###### 2020-10-27 화요일
###### 2020-10-29 목요일

In [1]:
# 워닝옵션 제거

import warnings
warnings.filterwarnings(action='ignore')

# 06_정수인코딩(Integer Encoding)
   - 컴퓨터틑 텍스트보다 숫자를 처리하는 것이 익숙하다.
   - 그러므로 자연어처리 또한, 텍스트를 숫자로 바꾸는 여러가지 기법들이 존재한다.
   - 그러한 기법들을 본격적으로 적용시키기 위한 첫 단계로 각 단어들을 고유한 정수에 **맵핑(mapping)**시키는 전처리 작업이 필요하다
   - 예를들어 가지고 있는 텍스트에 단어가 5,000개라면, 1번부터 5,000번까지 단어와 맵핑되는 고유한 정수(인덱스)를 부여한다.
   - 정수(인덱스)를 부여하는 방법은 여러가지가 있을 수 있는데, **보통 빈도수가 높은 단어들만 사용하기 위해서 단어에 대한 빈도수를 기준으로 정렬한 뒤에 정수(인덱스)를 부여한다.**

### 목차

#### 6-1 정수인코딩(Integer Encoding)

#### 6-2 케라스(Keras)의 텍스트 전처리

## 6-1 정수인코딩(Integer Encoding)

   - 왜 이러한 작업이 필요한지는 뒤에서 `원핫인코딩`과 `워드임베딩` 챕터에서 알아보자
   - 어떤 과정으로 단어에 정수 인덱스를 부여하는지 알아보자
   
   
   
##### 단어를 빈도수 순으로 정렬한 `단어집합(Vocabulary)`을 만들고, 빈도수가 높은 순서에서 낮은 순서로 정수 인덱스를 부여하는 방법이 있다.

### (1) dictionary 사용하는 방법

##### 정수 인코딩을 위해 실습을 진행해 보자
 > 아래 실습은 개발자의 실력에 따라 하나의 이 중 for문으로 처리할 수 있지만,
 >
 > 나는 처음 자연어처리를 배우는 과정이기에 하나하나 끊어서 천천히 진행할 것이다.
   1. `text`로 부터 **문장 토큰화**를 진행한다
   2. **문장 토큰**으로 부터 **단어토큰화**를 진행한다.
   3. **단어 토큰**에서 **불용어**를 처리한다. 처리하고 `result_tokens`리스트에 담는다.
   4. **단어사전(vocab)**을 만들자
   5. **단어사전**에 근거하여 빈도수가 높은 단어부터 **정수인덱스**를 부여한다
   6. 빈도수 **상위 5개**의 단어만 추출하고 `top5_vocab` 딕셔너리에 저장하라
   7. `result_tokens`리스트에 `top5_vocab`를 사용하여 문자로 이루어진 단어들을 숫자로 인코딩하라.

In [2]:
from nltk.tokenize import sent_tokenize  # 문장 토큰화
from nltk.tokenize import word_tokenize  # 단어 토큰화
from nltk.corpus import stopwords

In [3]:
text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

 1. `text`로 부터 **문장 토큰화**를 진행한다

In [4]:
sentence_tokens = sent_tokenize(text)
print(sentence_tokens)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


2. **문장 토큰**으로 부터 **단어토큰화**를 진행한다.

In [5]:
# 단어 토큰을 담는 리스트
word_tokens =[]

# 문장 토큰에서 단어 토큰화 진행하기
for token in sentence_tokens:
    temp_list = list(word_tokenize(token))
    word_tokens.append(temp_list)
    
print(word_tokens)

[['A', 'barber', 'is', 'a', 'person', '.'], ['a', 'barber', 'is', 'good', 'person', '.'], ['a', 'barber', 'is', 'huge', 'person', '.'], ['he', 'Knew', 'A', 'Secret', '!'], ['The', 'Secret', 'He', 'Kept', 'is', 'huge', 'secret', '.'], ['Huge', 'secret', '.'], ['His', 'barber', 'kept', 'his', 'word', '.'], ['a', 'barber', 'kept', 'his', 'word', '.'], ['His', 'barber', 'kept', 'his', 'secret', '.'], ['But', 'keeping', 'and', 'keeping', 'such', 'a', 'huge', 'secret', 'to', 'himself', 'was', 'driving', 'the', 'barber', 'crazy', '.'], ['the', 'barber', 'went', 'up', 'a', 'huge', 'mountain', '.']]


3. **단어 토큰**에서 **불용어**를 처리한다. 처리하고 `result_tokens`리스트에 담는다.

In [6]:
# nltk에서 영어 불용어 가져오기
stop_words = set(stopwords.words('english'))


# 불용어를 처리하자
result_tokens = []
for tokens in word_tokens:
    temp_list = []
    for token in tokens :
        token = token.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄인다.
        if (token not in stop_words) & (len(token) > 2) : # 단어토큰이 불용어에 없거나, 길이가 2이하인 단어는 제거한다.
            temp_list.append(token)
    result_tokens.append(temp_list)
    
    
print(result_tokens)


[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


4. **단어사전(vocab)**을 만들자

In [7]:
vocab = {}

for tokens in result_tokens:
    for token in tokens:
        if token not in vocab.keys():
            vocab[token] = 0
        vocab[token] += 1   
        
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


5. **단어사전**에 근거하여 빈도수가 높은 단어부터 **정수인덱스**를 부여한 **단어인덱스사전(index_vocab)**을 만들자

In [8]:
# 단어사전(vocab)의 value를 기준으로 정렬하자
sort_vocab = dict(sorted(vocab.items(), key=lambda x : x[1], reverse=True))
print('vocab을 value기준으로 정렬')
print(sort_vocab)
print()

index_vocab = {}


# vocab의 빈도수를 기준으로 정수인덱스를 부여하자
# 단어빈도가 낮은 단어는 제거한다 (정제 : cleaning)

idx = 1
for key, value in sort_vocab.items():
    if value > 1: # chapter02 정제 : 빈도가 낮은 단어는 제거한다! 기억해요
        index_vocab[key] = idx
        idx += 1
    

print('index_vocab')    
print(index_vocab)

vocab을 value기준으로 정렬
{'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}

index_vocab
{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


   - 단어 빈도수가 큰 단어부터 정수 인덱스를 부여한 ***단어인덱스사전**을 만드는 과정에서 **빈도수가 작은 단어**를 제거하였다
   - 빈도수가 작은단어는 자연어처리에서 의미를 가지지 않는 가능성이 높기때문에 제거하였다. 자세한건 `chapter02 정제 and 정규화`를 참조하자

6. 빈도수 **상위 5개**의 단어만 추출하고 `top5_vocab` 딕셔너리에 저장하라
   - 위말을 다르게 말하면 인덱스 5번까지 추출하라는 말과 같다
   - 또한, top5_vocab의 key에 `OOV`라는 단어를 추가히고, 정수인덱스는 6을 부여한다.
   - 여기서 `OOV`는 `Out-Of-Vocabulary`의 약자로, 단어빈도 Top5에 속하지 않는 단어들은 모두 `OOV`의 정수인덱스 6을 부여할 것이다

In [9]:
# 빈도수 Top5에 속하는 단어 리스트 뽑기
top5_list = [word for word, index in index_vocab.items() if index < 6]


# index_vocab 딕셔너리에서 Top5에 속하는 단어만 추출한다.
top5_vocab = {}
for element in top5_list:
    top5_vocab[element] = index_vocab[element]

# key에 `OOV`를 추가하고 value는 6을 준다
top5_vocab['OOV'] = 6

print(top5_vocab)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


7. `result_tokens`리스트에 `top5_vocab`를 사용하여 문자로 이루어진 단어들을 숫자로 인코딩하라.

In [10]:
print('단어토큰화와 불용어 처리가 완료된 리스트')
print(result_tokens)
print('-'*130)

print()
print('빈도수가 높은 순서부터 인댁스를 부여하고 빈도수 상위 5개만 담은 딕셔너리')
print(top5_vocab)
print()

단어토큰화와 불용어 처리가 완료된 리스트
[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
----------------------------------------------------------------------------------------------------------------------------------

빈도수가 높은 순서부터 인댁스를 부여하고 빈도수 상위 5개만 담은 딕셔너리
{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}



In [11]:
frequency_tokens = []

for tokens in result_tokens:
    temp_list = []
    for token in tokens:
        if token in top5_vocab.keys():
            temp_list.append(top5_vocab[token])
        else :
            temp_list.append(top5_vocab['OOV'])
    frequency_tokens.append(temp_list)
    
print(frequency_tokens)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


##### 지금까지 파이썬의 딕셔너리 자료형으로 정수 인코딩을 진행하였다.
##### 지금부터는 조금 더 쉬운 방법들을 소개한다

### (2) Counter 사용하는 방법

In [12]:
from collections import Counter

##### 위에서 단어 토큰화 했던 `result_tokens`를 사용한다

In [13]:
print(result_tokens)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


 - 현재 `result_tokens`에는 리스트안에 리스트들이 포함되어있는 형태이다
##### 리스트안에 들어있는 리스트들을 하나의 리스트로 만들자

In [14]:
# 리스트안에 리스트를 flat하게 만들자

words = sum(result_tokens, [])
print(words)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


##### 이제 `collections` 패키지가 제공하는 `Counter` 함수를 사용해보자

In [15]:
vocab = Counter(words)
print(type(vocab))
print(vocab)

<class 'collections.Counter'>
Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


##### 위에서 만들어진 `vocab`에 ` most_common()`를 적용하여 최상위 빈도수를 가진 단어에서부터 수를 지정하여 지정한 수 만큼 단어를 가져올 수 있다.
   - 말이 어렵네유 ㅠㅠ.. 밑에 실습한번 보세요

In [16]:
print(vocab)
print(vocab.most_common(5))

# 수를 지정하여 상위 5개의 단어만 가져 왔다.

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})
[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


##### 이제 빈도수가 높은 단어 부터 낮은 정수인덱스를 부여하자

In [17]:
index_vocab = {}

idx = 1
for word, frequency in vocab.most_common(5):
    index_vocab[word] = idx
    idx += 1
    
print(index_vocab)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


##### 이하는 생략한다

### (3) NLTK의 FreqDist 사용하기

NLTK에서는 빈도수 계산 도구인 `FreqDist()`를 지원한다. `Counter()`와 같은 방법으로 사용할 수 있다.

In [18]:
from nltk import FreqDist
import numpy as np

 - 역시 토큰화와 불용어 처리가 완료된 `result_tokens`를 사용하자

In [19]:
# numpy의 np.hstack을 이용해서도 리스트를 flat화 할 수 있다.
np.hstack(result_tokens)

array(['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge',
       'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret',
       'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept',
       'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge',
       'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge',
       'mountain'], dtype='<U8')

In [20]:
vocab = FreqDist(np.hstack(result_tokens))
print(vocab)

<FreqDist with 13 samples and 36 outcomes>


In [21]:
vocab = vocab.most_common(5)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

 - 지금 까지의 결과는 `Counter()`를 사용하였을때와 같다
 - 다음과정은 컴프리헨션을 사용해보자

In [22]:
index_vocab = { word[0] : idx+1 for idx, word in enumerate(vocab) }
print(index_vocab)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


## 6-2 케라스(Keras)의 텍스트 전처리
   - 케라스는 기본적인 전처리를 위한 도구들을 제공한다.
   - 정수 인코딩을 위해 케라스의 토크나이저를 사용하기도 한다.
   - 실습해보자

위에서 토큰화와 불용어 처리를 완료한 `result_tokens`리스트가 존재하지만, 연습하는 김에 한번 더 실습해보자

In [26]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [25]:
text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

###### (1) 문장단위로 토큰화 : sent_tokenize()

In [29]:
sentences = sent_tokenize(text)
print(sentences)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


##### (2) 단어단위로 토큰화 : word_tokenize()

In [31]:
word_tokens = [word_tokenize(sentence) for sentence in sentences ]
print(word_tokens)

[['A', 'barber', 'is', 'a', 'person', '.'], ['a', 'barber', 'is', 'good', 'person', '.'], ['a', 'barber', 'is', 'huge', 'person', '.'], ['he', 'Knew', 'A', 'Secret', '!'], ['The', 'Secret', 'He', 'Kept', 'is', 'huge', 'secret', '.'], ['Huge', 'secret', '.'], ['His', 'barber', 'kept', 'his', 'word', '.'], ['a', 'barber', 'kept', 'his', 'word', '.'], ['His', 'barber', 'kept', 'his', 'secret', '.'], ['But', 'keeping', 'and', 'keeping', 'such', 'a', 'huge', 'secret', 'to', 'himself', 'was', 'driving', 'the', 'barber', 'crazy', '.'], ['the', 'barber', 'went', 'up', 'a', 'huge', 'mountain', '.']]


##### (3) 단어 토큰에서 불용어 처리

In [95]:
stop_words = set(stopwords.words('english'))

tokens = []

for words_list in word_tokens:
    temp_list = []
    for word in words_list:
        word = word.lower()
        if ( word not in stop_words ) & ( len(word)>2 ):
            temp_list.append(word)
    tokens.append(temp_list)

In [96]:
print(tokens)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


##### (4) `token`를 이용하여 케라스가 제공하는 `Tokenizer()`함수를 사용해보자

In [60]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [64]:
tokenizer = Tokenizer()

# 
tokenizer.fit_on_texts(tokens)


{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


 - `.fit_on_texts`는 빈도가 높은 순으로 정수 인코딩 작업을 진행해 주는 함수이다.
 - 정수인코딩의 결과를 보려면 `.word_index`를, 단어빈도수를 보려면 `.word_counts`를 사용하면 된다

In [66]:
print(tokenizer.word_counts)
print()
print(tokenizer.word_index)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


 - 단어 토큰화와 불용어 처리가 완료된 토큰에 정수인덱스를 부여하기위한 작업은 아래와 같다

In [67]:
print(tokenizer.texts_to_sequences(tokens))

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


##### (5) 케라스가 제공하는 함수인 `Tokenizer`를 이용해 빈도수가 높은 순으로 상위 5개의 단어만 정수인덱스를 부여하여라

In [73]:
tokenizer = Tokenizer(num_words = 5 + 1) # 상위 5개만 사용하겠다는 뜻

###### 위에서 `5+1`을 해준 이유는?
   - `num_words`는 숫자를 0부터 카운트한다.
   - 만약 5+1이 아닌 5를 입력하게 되면, 0 ~ 4번 단어를 보존하겠다는 의미이다.
   - fit_on_texts는 인덱스를 1번부터 부여하기 때문에 5를입력하면 1, 2, 3, 4 번까지만 정수인덱스를 부여하게된다.
   - 그래서 5가아닌 5+1을 부여한 것이다.
   
   
###### 그렇다면 왜 `num_words`는 0부터 카운트를 하고, `.fit_on_text`는 1부터 정수인덱스를 부여합니까!
   - 자연어 처리에서 **패딩(Padding)**이라는 작업때문이다.
   - 이것은 **패딩**을 다룰때, 다시 설명하겠다.

In [74]:
tokenizer.fit_on_texts(tokens)

In [75]:
print(tokenizer.word_index)
print(tokenizer.word_counts)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}
OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


##### 위 결과는 상위 5개만 출력해야 하는거 아닙니꺼..?
   - 정수 `.word_index`와 `.word_counts`는 모든 단어에 대해 수행한다.
   - 상위 5개만 적용되는 결과는 다음 코드에서 나타날 것이다

In [77]:
print(tokenizer.texts_to_sequences(tokens))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


 - 위 결과를 보면 인덱스가 1, 2, 3, 4, 5로 5가지임을 볼 수있다.
 - 상위 5개에 포함되어 있지 않으면, 정수 인덱스를 부여할 수 없기 때문에, 상위 6번째 이후의 단어는 삭제 된다.

##### (7) 빈도수가 상위 5개의 단어만 정수인덱스(1, 2, 3, 4, 5)를 부여하고, 상위 5에 들지않는 단어들은 정수인덱스 6을 부여하여라

 1. **num_words를 지정하지 않는 방법**

In [114]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokens)

index_vocab = {key : index for key, index in tokenizer.word_index.items() if index < 6}
index_vocab['OOV'] = 6
print(index_vocab)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


In [124]:
flat_tokens = sum(tokens, [])


index_tokens = []
for token_list in tokens :
    temp_list = []
    for token in token_list:
        if token in index_vocab.keys():
            temp_list.append(index_vocab[token])
        else:
            temp_list.append(index_vocab['OOV'])
    index_tokens.append(temp_list)


[[1, 5],
 [1, 6, 5],
 [1, 3, 5],
 [6, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [6, 6, 3, 2, 6, 1, 6],
 [1, 6, 3, 6]]

In [125]:
print(index_tokens)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


 2. **num_words를 지정하는 방법**

In [127]:
tokenizer = Tokenizer(num_words = 5 + 2,   
                      oov_token = 'OOV')

In [128]:
tokenizer.fit_on_texts(tokens)

###### `Tokenizer()`함수는 기본적으로 `OOV`를 지정할 수 있는데, `OOV`의 인덱스는 1이다.
   - 위에서 했던 방법들은 마지막 인덱스인 6을 `OOV`이 인덱스로 부여하였지만, 
   - `Tokenizer()`함수는 위의 방법들과 다르게 `OOV`의 정수인덱스는 1을 부여한다.

In [130]:
print(tokenizer.word_index)

{'OOV': 1, 'barber': 2, 'secret': 3, 'huge': 4, 'kept': 5, 'person': 6, 'word': 7, 'keeping': 8, 'good': 9, 'knew': 10, 'driving': 11, 'crazy': 12, 'went': 13, 'mountain': 14}
OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [131]:
tokenizer.texts_to_sequences(tokens)

[[2, 6],
 [2, 1, 6],
 [2, 4, 6],
 [1, 3],
 [3, 5, 4, 3],
 [4, 3],
 [2, 5, 1],
 [2, 5, 1],
 [2, 5, 3],
 [1, 1, 4, 3, 1, 2, 1],
 [2, 1, 4, 1]]